<a href="https://colab.research.google.com/github/harjeet88/llm-course/blob/main/projects/1_robo_interviewer_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -q gradio google-generativeai pydantic

In [94]:
import gradio as gr
import sqlite3
import os
import google.generativeai as genai
from pydantic import BaseModel
from typing import List, Tuple
import uuid
import json
import re

In [95]:
# Set up logging
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [96]:
# In-memory state store
state_store = {}

In [97]:
# Pydantic models for data validation
class InterviewConfig(BaseModel):
    num_questions: int
    technology: str
    candidate_id: str
    difficulty: str

class InterviewResponse(BaseModel):
    question: str
    answer: str
    score: int
    feedback: str

In [98]:
# Database setup
def init_db():
    conn = sqlite3.connect('interview.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS interviews
                 (interview_id TEXT PRIMARY KEY, candidate_id TEXT, config TEXT, start_time TEXT)''')
    c.execute('''CREATE TABLE IF NOT EXISTS responses
                 (response_id TEXT PRIMARY KEY, interview_id TEXT, question TEXT, answer TEXT, score INTEGER, feedback TEXT)''')
    conn.commit()
    conn.close()

In [99]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GEMINI_API_KEY=GOOGLE_API_KEY

In [100]:
# Gemini API setup
def configure_gemini():
    api_key = GEMINI_API_KEY
    if not api_key:
        raise ValueError("GEMINI_API_KEY environment variable not set.")
    genai.configure(api_key=api_key)
    return genai.GenerativeModel('gemini-2.5-flash')

In [101]:
# Clean raw response to extract JSON
def clean_json_response(raw_text: str) -> str:
    cleaned = re.sub(r'^```json\n|```$', '', raw_text).strip()
    cleaned = re.sub(r',\s*}', '}', cleaned)
    cleaned = re.sub(r',\s*\]', ']', cleaned)
    return cleaned

In [102]:
def get_previous_questions(interview_id: str) -> List[str]:
    conn = sqlite3.connect('interview.db')
    c = conn.cursor()
    c.execute("SELECT question FROM responses WHERE interview_id = ?", (interview_id,))
    questions = [row[0] for row in c.fetchall()]
    conn.close()
    return questions

In [103]:
def generate_question(model, technology: str, difficulty: str, question_number: int, interview_id: str) -> Tuple[str, str]:
    previous_questions = get_previous_questions(interview_id)
    previous_questions_str = "\n".join(previous_questions) if previous_questions else "None"
    prompt = f"""
    Generate a {technology} interview question ({difficulty} level) with a clear answer key. Ensure the question is different from the following previously asked questions:
    {previous_questions_str}
    Format as JSON: {{'question': '...', 'answer_key': '...'}}
    """
    max_attempts = 3
    for attempt in range(max_attempts):
        try:
            response = model.generate_content(prompt)
            logger.debug(f"Raw question response (attempt {attempt + 1}): {response.text}")
            cleaned_response = clean_json_response(response.text)
            result = json.loads(cleaned_response)
            question = result['question']
            if question not in previous_questions:
                return question, result['answer_key']
            logger.debug(f"Question repeated: {question}, retrying...")
        except Exception as e:
            logger.error(f"Error generating question (attempt {attempt + 1}): {e}")
            if attempt == max_attempts - 1:
                fallback_question = f"Explain a unique {technology} concept ({difficulty} level) not previously covered."
                return fallback_question, f"A clear explanation of a unique {technology} concept."
    return fallback_question, f"A clear explanation of a unique {technology} concept."

In [104]:
# Evaluate response using Gemini
def evaluate_response(model, question: str, answer: str, answer_key: str) -> Tuple[int, str]:
    prompt = f"""
    Evaluate the following candidate response for correctness, clarity, and completeness on a scale of 0-100.
    Question: {question}
    Candidate Response: {answer}
    Reference Answer: {answer_key}
    Return JSON: {{'score': int, 'feedback': 'string'}}
    """
    try:
        response = model.generate_content(prompt)
        result = json.loads(response.text.strip('```json\n').strip('```'))
        return result['score'], result['feedback']
    except Exception as e:
        logger.error(f"Error evaluating response: {e}")
        return 50, f"Evaluation failed: {str(e)}. Default score assigned."

In [105]:
# Store interview and response data
def save_interview(candidate_id: str, config: dict) -> str:
    interview_id = str(uuid.uuid4())
    conn = sqlite3.connect('interview.db')
    c = conn.cursor()
    c.execute("INSERT INTO interviews (interview_id, candidate_id, config, start_time) VALUES (?, ?, ?, datetime('now'))",
              (interview_id, candidate_id, json.dumps(config)))
    conn.commit()
    conn.close()
    return interview_id


In [106]:
def save_response(interview_id: str, question: str, answer: str, score: int, feedback: str):
    response_id = str(uuid.uuid4())
    conn = sqlite3.connect('interview.db')
    c = conn.cursor()
    c.execute("INSERT INTO responses (response_id, interview_id, question, answer, score, feedback) VALUES (?, ?, ?, ?, ?, ?)",
              (response_id, interview_id, question, answer, score, feedback))
    conn.commit()
    conn.close()

In [107]:
# Get final score
def get_final_score(interview_id: str, total_questions: int) -> Tuple[float, List[InterviewResponse]]:
    conn = sqlite3.connect('interview.db')
    c = conn.cursor()
    c.execute("SELECT question, answer, score, feedback FROM responses WHERE interview_id = ?", (interview_id,))
    responses = [InterviewResponse(question=row[0], answer=row[1], score=row[2], feedback=row[3]) for row in c.fetchall()]
    conn.close()
    if responses:
        final_score = sum(r.score for r in responses) / total_questions  # Divide by total questions, not answered
        return final_score, responses
    return 0, []

In [108]:
# Gradio interface logic
def start_interview(num_questions: int, technology: str, difficulty: str, candidate_id: str):
    logger.debug(f"Starting interview: num_questions={num_questions}, technology={technology}, difficulty={difficulty}, candidate_id={candidate_id}")
    if num_questions < 1 or num_questions > 10:
        return "Number of questions must be between 1 and 10.", [], "", None
    if not candidate_id:
        return "Please provide a candidate ID.", [], "", None
    if difficulty not in ["Beginner", "Intermediate", "Expert"]:
        return "Invalid difficulty level.", [], "", None

    try:
        model = configure_gemini()
        config = {"num_questions": num_questions, "technology": technology, "difficulty": difficulty}
        interview_id = save_interview(candidate_id, config)
        question, answer_key = generate_question(model, technology, difficulty, 1, interview_id)
        state = {"interview_id": interview_id, "current_question": 1, "answer_key": answer_key, "model": model, "config": config}
        state_store[interview_id] = state
        logger.debug(f"Interview started: {interview_id}, first question: {question}")
        return question, [(question, None)], interview_id, state
    except ValueError as e:
        logger.error(f"Error starting interview: {e}")
        return str(e), [], "", None

In [109]:
def submit_response(message: str, history: List[Tuple[str, str]], interview_id: str, state):
    logger.debug(f"Submitting response: message={message}, history={history}, interview_id={interview_id}, state={state}")

    if not state or not isinstance(state, dict):
        logger.debug(f"State missing or invalid, attempting to retrieve from store with interview_id: {interview_id}")
        state = state_store.get(interview_id)
        if not state:
            logger.error(f"No state found for interview_id: {interview_id}")
            return "Interview not started or session expired. Please start a new interview.", history, interview_id, None

    interview_id = state.get("interview_id")
    current_question = state.get("current_question")
    model = state.get("model")
    config = state.get("config")
    answer_key = state.get("answer_key")

    if not all([interview_id, current_question, model, config, answer_key]):
        logger.error(f"Invalid state contents: {state}")
        return "Invalid interview state. Please start a new interview.", history, interview_id, None

    score, feedback = evaluate_response(model, history[-1][0], message, answer_key)
    save_response(interview_id, history[-1][0], message, score, feedback)
    history[-1] = (history[-1][0], message)

    if current_question < config["num_questions"]:
        next_question, next_answer_key = generate_question(model, config["technology"], config["difficulty"], current_question + 1, interview_id)
        state["current_question"] = current_question + 1
        state["answer_key"] = next_answer_key
        state_store[interview_id] = state
        history.append((next_question, None))
        logger.debug(f"Next question generated: {next_question}")
        return f"Answer submitted. Please answer the next question.", history, interview_id, state
    else:
        final_score, responses = get_final_score(interview_id, config["num_questions"])
        feedback_summary = "\n".join([f"Q: {r.question}\nA: {r.answer}\nScore: {r.score}\nFeedback: {r.feedback}" for r in responses])
        logger.debug(f"Interview completed: final_score={final_score}")
        state_store.pop(interview_id, None)
        return f"Interview completed! Final Score: {final_score:.2f}/100\n\nSummary:\n{feedback_summary}", history, "", None


In [110]:
def end_interview(interview_id: str, history: List[Tuple[str, str]], state):
    logger.debug(f"Ending interview: interview_id={interview_id}, state={state}")

    # Retrieve state from store if not provided
    if not state or not isinstance(state, dict):
        logger.debug(f"State missing or invalid, attempting to retrieve from store with interview_id: {interview_id}")
        state = state_store.get(interview_id)
        if not state:
            logger.error(f"No state found for interview_id: {interview_id}")
            return "Interview not started or session expired.", history, interview_id, None

    # Validate state contents
    interview_id = state.get("interview_id")
    config = state.get("config")

    if not all([interview_id, config]):
        logger.error(f"Invalid state contents: {state}")
        return "Invalid interview state.", history, interview_id, None

    # Calculate final score based on all questions
    final_score, responses = get_final_score(interview_id, config["num_questions"])
    feedback_summary = "\n".join([f"Q: {r.question}\nA: {r.answer}\nScore: {r.score}\nFeedback: {r.feedback}" for r in responses])
    logger.debug(f"Interview ended early: final_score={final_score}")
    # Clear state from store
    state_store.pop(interview_id, None)
    return f"Interview ended early! Final Score: {final_score:.2f}/100\n\nSummary:\n{feedback_summary}", history, "", None

In [111]:
# Initialize database
init_db()

In [112]:
# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## Robo Interviewer System")
    candidate_id = gr.Textbox(label="Candidate ID")
    num_questions = gr.Slider(minimum=1, maximum=10, step=1, label="Number of Questions", value=5)
    technology = gr.Dropdown(choices=["Python", "Java", "DevOps", "JavaScript"], label="Technology", value="Python")
    difficulty = gr.Dropdown(choices=["Beginner", "Intermediate", "Expert"], label="Difficulty Level", value="Intermediate")
    interview_id = gr.Textbox(label="Interview ID", visible=False)  # Hidden field for interview_id
    start_btn = gr.Button("Start Interview")
    end_btn = gr.Button("End Interview")
    chatbot = gr.Chatbot(label="Interview Chat")
    msg = gr.Textbox(label="Your Answer")
    output = gr.Textbox(label="Feedback and Score")

    start_btn.click(
        fn=start_interview,
        inputs=[num_questions, technology, difficulty, candidate_id],
        outputs=[output, chatbot, interview_id, gr.State()]
    )
    msg.submit(
        fn=submit_response,
        inputs=[msg, chatbot, interview_id, gr.State()],
        outputs=[output, chatbot, interview_id, gr.State()]
    )
    end_btn.click(
        fn=end_interview,
        inputs=[interview_id, chatbot, gr.State()],
        outputs=[output, chatbot, interview_id, gr.State()]
    )

/tmp/ipython-input-112-1105992436.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Interview Chat")


In [113]:
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bd225d3928e7b23a26.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:__main__:Error evaluating response: Expecting value: line 1 column 1 (char 0)
ERROR:__main__:Error evaluating response: Expecting value: line 1 column 1 (char 0)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://bd225d3928e7b23a26.gradio.live
